## Install Dependencies

In [ ]:
!pip install langchain==0.3.4
!pip install langchain-openai==0.2.3
!pip install langchain-community==0.3.3
!pip install jq==1.8.0
!pip install httpx==0.27.2
# install vectordb and bm25 utils
!pip install langchain-chroma==0.1.4
!pip install rank_bm25==0.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 27.3 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.6
    Uninstalling langsmith-0.3.6:
      Successfully uninstalled langsmith-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found ex

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 7.5 MB/s eta 0:00:00


## Setup Environment Variables

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "API_KEY"

## Get the Dataset


In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
docs[1]

Document(metadata={'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf', 'page': 1}, page_content=' \na) os detentores de mandato eletivo; \nb) os ocupantes de cargos vitalícios; \nc) os ocupantes de cargos de direção ou de chefia. \n§ 2º Pelo menos cinquenta por cento dos cargos em comissão devem ser \nprovidos por servidor público de carreira, nos casos e condições previstos em lei. \n§ 3º É proibid a a designação para função de confiança ou a nomeação para \ncargo em comissão, incluídos os de natureza especial, de pessoa que tenha praticado \nato tipificado como causa de inelegibilidade prevista na legislação eleitoral, observado o \nmesmo prazo de incompatibilidade dessa legislação. \nArt. 6º  As funções de confiança, privativas de servidor efetivo, destinam -se \nexclusivamente às atribuições de direção, chefia e assessoramento. \nArt. 7º São requisitos básicos para investidura em cargo público:  \nI – a nacionalidade brasileira; \nII – o gozo dos direitos políticos

### Splitting Document Text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=0) #default separator = ["\n\n", "\n", " ", ""]

all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

73

## Create Vector Database Index and Setup Semantic Retrieval

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')
# create vector DB of docs and embeddings - takes < 30s on Colab - LEVOU 7 SEGUNDOS
chroma_db = Chroma.from_documents(documents=all_splits,
                                  collection_name='my_context_db',
                                  embedding=openai_embed_model,
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./my_context_db")

In [ ]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

## Create BM25 Index and Setup Keyword Retrieval

In [ ]:
from langchain.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents=docs,k=5)

## Enable Hybrid Search with Ensemble Retrieval

In [ ]:
from langchain.retrievers import EnsembleRetriever
# reciprocal rank fusion
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, similarity_retriever],
    weights=[0.5, 0.5]
)

## Improving Retriever with Reranker

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

# download an open-source reranker model - BAAI/bge-reranker-v2-m3
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
reranker_compressor = CrossEncoderReranker(model=reranker, top_n=3)
# Retriever 2 - Uses a Reranker model to rerank retrieval results from the previous retriever
final_retriever = ContextualCompressionRetriever(
    base_compressor=reranker_compressor,
    base_retriever=ensemble_retriever
)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

## Testing our Retrieval Pipeline

In [ ]:
from IPython.display import display, Markdown
def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content[:1000]))
        print()
query = "O que é cargo público?"
top_docs = final_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'page': 1, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf'}
Content Brief:


 
a) os detentores de mandato eletivo; 
b) os ocupantes de cargos vitalícios; 
c) os ocupantes de cargos de direção ou de chefia. 
§ 2º Pelo menos cinquenta por cento dos cargos em comissão devem ser 
providos por servidor público de carreira, nos casos e condições previstos em lei. 
§ 3º É proibid a a designação para função de confiança ou a nomeação para 
cargo em comissão, incluídos os de natureza especial, de pessoa que tenha praticado 
ato tipificado como causa de inelegibilidade prevista na legislação eleitoral, observado o 
mesmo prazo de incompatibilidade dessa legislação. 
Art. 6º  As funções de confiança, privativas de servidor efetivo, destinam -se 
exclusivamente às atribuições de direção, chefia e assessoramento. 
Art. 7º São requisitos básicos para investidura em cargo público:  
I – a nacionalidade brasileira; 
II – o gozo dos direitos políticos; 
III – a quitação com as obrigações militares e eleitorais; 
IV – o nível de escolaridade exigido para o exercício do cargo; 



Metadata: {'page': 1, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf'}
Content Brief:


a) os detentores de mandato eletivo; 
b) os ocupantes de cargos vitalícios; 
c) os ocupantes de cargos de direção ou de chefia. 
§ 2º Pelo menos cinquenta por cento dos cargos em comissão devem ser 
providos por servidor público de carreira, nos casos e condições previstos em lei. 
§ 3º É proibid a a designação para função de confiança ou a nomeação para 
cargo em comissão, incluídos os de natureza especial, de pessoa que tenha praticado 
ato tipificado como causa de inelegibilidade prevista na legislação eleitoral, observado o 
mesmo prazo de incompatibilidade dessa legislação. 
Art. 6º  As funções de confiança, privativas de servidor efetivo, destinam -se 
exclusivamente às atribuições de direção, chefia e assessoramento. 
Art. 7º São requisitos básicos para investidura em cargo público:  
I – a nacionalidade brasileira; 
II – o gozo dos direitos políticos; 
III – a quitação com as obrigações militares e eleitorais; 
IV – o nível de escolaridade exigido para o exercício do cargo; 
V 


Metadata: {'page': 10, 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf'}
Content Brief:


 
quando houver compatibilidade de horários, para: 
I – dois cargos de professor; 
II – um cargo de professor com outro técnico ou científico; 
III – dois cargos ou empregos privativos de profissionais de saúde,  com 
profissões regulamentadas. 
§ 1º Presume-se como cargo de natureza técnica ou científica, para os fins do 
inciso II, qualquer cargo público para o qual se exija educação superior ou educação 
profissional, ministrada na forma e nas condições previstas na Lei de Diretrizes e Bases 
da Educação Nacional.  
§ 2º A proibição de acumular estende-se:  
I – a empregos e funções e abrange autarquias, fundações, empresas públicas, 
sociedades de economia mi sta, suas subsidiárias e sociedades controladas direta ou 
indiretamente pelo poder público;  
II – aos proventos de aposentadoria pagos por regime próprio de previdência 
social do Distrito Federal, da União, de Estado ou Município, ressalvados os proventos 
decorrentes de cargo acumulável na forma deste artigo.  
§ 3º O serv

In [ ]:
query = "Como servidora pública do GDF, faço jus a quais tipos de licença?"
top_docs = final_retriever.invoke(query)
display_docs(top_docs)

NameError: name 'final_retriever' is not defined

##Building our Contextual RAG Pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """Você é um assistente altamente especializado em responder perguntas com base em informações fornecidas.
                Utilize exclusivamente o contexto recuperado para formular sua resposta.

                - Se a resposta não estiver no contexto, não tente adivinhar ou inventar, apenas informe que não possui essa informação.
                - Estruture sua resposta de forma clara, objetiva e bem formatada, mantendo o máximo de detalhes relevantes.
                - Responda em Português.
                - Cite os artigos em que as respostas foram baseadas.

                Pergunta:
                {question}

                Contexto:
                {context}

                Resposta:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [ ]:
from langchain_openai import ChatOpenAI
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = ({
    "context": (final_retriever
                |
                format_docs),
    "question": RunnablePassthrough()
    }
                |
                rag_prompt_template
                |
                chatgpt)

In [ ]:
from IPython.display import display, Markdown
query = "Como servidora pública do GDF, faço jus a quais tipos de licença?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Como servidora pública do GDF, você faz jus aos seguintes tipos de licença, conforme estabelecido no Art. 130:

1. **Licença por motivo de afastamento do cônjuge ou companheiro**;
2. **Licença por motivo de doença em pessoa da família**;
3. **Licença para o serviço militar**;
4. **Licença para atividade política**;
5. **Licença prêmio por assiduidade**;
6. **Licença para tratar de interesses particulares**;
7. **Licença para desempenho de mandato classista**;
8. **Licença de paternidade**;
9. **Licença de maternidade**;
10. **Licença médica ou odontológica**.

Além disso, é importante observar que a concessão da licença-maternidade está sujeita às normas do regime de previdência social ao qual a servidora está filiada (Art. 130, parágrafo único).

Para mais detalhes sobre a concessão e prorrogação das licenças, consulte os artigos 131 e 132, que tratam da prorrogação de licenças e do direito de retorno à mesma lotação após o término das licenças. 

Referências: 
- Art. 130
- Art. 131
- Art. 132

In [ ]:
query = "O que é um cargo público?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Um cargo público é uma posição ocupada por um servidor que desempenha funções em uma entidade pública, sendo regido por normas específicas. Os requisitos básicos para a investidura em um cargo público incluem:

1. Nacionalidade brasileira;
2. Gozo dos direitos políticos;
3. Quitação com as obrigações militares e eleitorais;
4. Nível de escolaridade exigido para o exercício do cargo;
5. Idade mínima de dezoito anos;
6. Aptidão física e mental.

Esses requisitos devem ser comprovados no momento da posse (Art. 7º).

As formas de provimento de um cargo público incluem: nomeação, reversão, aproveitamento, reintegração e recondução (Art. 8º). Além disso, o exercício do cargo público se refere ao efetivo desempenho das atribuições atribuídas ao servidor (Art. 19).

Os cargos públicos podem ser ocupados por servidores efetivos, que são aqueles que passaram por concurso público, e também por cargos em comissão, que são de livre nomeação e exoneração (Art. 10).

Para mais detalhes, consulte os artigos mencionados, especialmente o Art. 7º, Art. 8º e Art. 19.

## Evaluation Metrics

In [ ]:
memory.clear()

In [ ]:
!pip install datasets
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
questions = ["Qual é o percentual aplicado pela Lei Complementar 840/2011 para reserva de vagas às pessoas portadoras de deficiência?",
             "A realização de concurso público é obrigatório para que?",
             "Quais fatores são observados durante o estágio probatório de um servidor público?",
             "Tenho direito a folga para consulta ou exames de rotina?",
             "Para fazer mestrado ou doutorado, tenho direito a horário diferenciado ou licença?",
             "Fui aprovado em concurso público federal, tendo direito a licença remunerada?",
             "O que acontece se eu acumular dois cargos públicos?",
             "Estou próximo a me aposentar, tenho alguma restrição?",
             "Posso ter um outro trabalho na esfera privada?",
             "Como é a licença prêmio/servidor?"
            ]

ground_truths = [["O percentual aplicado para reserva de vagas às pessoas portadoras de deficiência é de 20%"],
                ["A realização de concurso público é obrigatório para provimento de cargo efetivo e emprego público"],
                ["São observadas a responsabilidade, a assiduidade, a capacidade de iniciativa e a produditividades."],
                ["Sem prejuízo da remuneração ou subsídio, o servidor pode ausentar-se do serviço, mediante comunicação prévia à chefia imediata: por um dia para realizar, uma vez por ano, exames médicos preventivos ou periódicos voltados ao controle de câncer de próstata, de mama ou do colo de útero"],
                ["Pode ser concedido horário especial ao servidor matriculado em curso da educação básica e da educação superior, quando comprovada a incompatibilidade entre o horário escolar e o da unidade administrativa, sem prejuízo do exercício do cargo. É exigida do servidor a compensação de horário na unidade administrativa, de modo a cumprir integralmente o regime semanal de trabalho. O servidor estudante deve comprovar, mensalmente, a sua frequência escolar. Sobre o afastamento para participar de programa de Pós-Graduação Strictu Sensu, o servidor estável pode, no interesse da administração pública, e desde que a partici­pação não possa ocorrer simultaneamente com o exercício do cargo ou mediante compensação horário, afastar-se do exercício do cargo efetivo, com a respectiva remuneração ou subsídio, para participar de programa de pós-graduação stricto sensu em instituição de ensino superior, no País ou no exterior. O afastamento para realização de programas de mestrado, doutorado ou pós-doutorado somente pode ser concedido ao servidor estável que esteja em efetivo exercício no respectivo órgão, autarquia ou fundação há pelo menos três anos consecutivos para mestrado e quatro anos consecutivos para doutorado ou pós-doutorado."],
                ["Pode gozar de afastamento caso haja expressa previsão do curso de formação no edital do concurso e incompatibilidade entre os horários das aulas e os da repartição. O afastamento só pode ser com remuneração ou subsídio, nos casos de curso de formação para cargo efetivo de órgão, autarquia ou fundação dos Poderes Legislativo ou Executivo do Distrito Federal."],
                ["A Lei Complementar nº 840/2011 permite a acumulação de cargos públicos apenas nas hipóteses previstas na Constituição Federal (art. 37, XVI), como:a) dois cargos de professor; b) um cargo de professor e outro técnico ou científico; c) dois cargos ou empregos privativos de profissionais de saúde. Se um servidor acumular cargos de forma irregular, ele pode ser punido com demissão. O processo ocorre por meio de um PAD, que assegura o direito ao contraditório e à ampla defesa antes da aplicação da penalidade."],
                ["Algumas possíveis restrições: a) é necessário que o servidor tenha completado o tempo de contribuição e a idade exigidos por lei para se aposentar voluntariamente; b) apenas servidores efetivos podem solicitar a aposentadoria voluntária, c) se um servidor estiver respondendo a um Processo Administrativo Disciplinar (PAD) que possa resultar em demissão, seu pedido de aposentadoria poderá ser negado até a conclusão do processo; d) caso o servidor tenha sofrido penalidades graves, como demissão por infração disciplinar, pode perder o direito à aposentadoria estatutária; e) se o servidor tiver se ausentado por afastamento para estudo ou missão no exterior ou para participar de programa de Pós-Graduação Stricto Sensu e aposentar antes de decorrido período igual ao do afastamento, o servidor de ressarcir proporcionalmente a despesa, incluída a remuneração ou o subsídio e encargos sociais."],
                ["A Lei Complementar nº 840/2011 não proíbe o exercício de atividade remunerada no setor privado, desde que não haja conflito de interesses com o cargo público e que a atividade não comprometa a jornada de trabalho do servidor. São consideradas infrações: exercer atividade privada incompatível com o horário do serviço; exercer atividade privada incompatível com o exercício do cargo público ou da função de confiança; participar de gerência ou administração de sociedade ou empresa privada, personificada ou não personificada, salvo: a) nos casos previstos nesta Lei Complementar; b) nos períodos de licença ou afastamento do cargo sem remuneração, desde que não haja proi­bição em sentido contrário, nem incompatibilidade; c) em instituições ou entidades beneficentes, filantrópicas, de caráter social e humanitário e sem fins lucrativos, quando compatíveis com a jornada de trabalho."],
                ["A licença prêmio foi substituída pela licença servidor pela Lei Complementar 952/2019. Após cada quinquênio de efetivo exercício, o servidor ocupante de cargo efetivo faz jus a 3 meses de licença-servidor, sem prejuízo de sua remuneração, inclusive da retribuição do cargo em comissão, função de confiança ou função gratificada escolar - FGE que eventualmente exerça. Os períodos não são acumuláveis e não podem ser convertidos em pecúnia. O número de servidores afastados em virtude de licença-servidor não pode ser superior a 1/3 da lotação da respectiva unidade administrativa do órgão, autarquia ou fundação. A administração tem o prazo de até 120 dias, contado da data de requerimento do pedido pelo servidor, para definir o período de gozo da licença. Fica assegurado às servidoras e aos servidores o direito de iniciar a fruição de licença-servidor logo após o término da licença-maternidade ou da licença-paternidade."]]

answers = []
contexts = []
references = []

# Inference
# Inference
for query in questions:
  # Extract the generated text from the LLM response
  response = qa_rag_chain.invoke(query)
  # Access content using .content attribute
  answers.append(response.content)
  contexts.append([docs.page_content for docs in final_retriever.invoke(query)])
  # Add the ground truth as the reference here
  ground_truth_for_query = ground_truths[questions.index(query)]
  references.append(ground_truth_for_query)

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    # **Remove 'ground_truths', keep 'reference' as a string**
    "reference": [item[0] for item in ground_truths]  # Extract the string from the list
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.3/178.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,Qual é o percentual aplicado pela Lei Compleme...,[Art. 11. As normas gerais sobre concurso púb...,O percentual aplicado pela Lei Complementar 84...,O percentual aplicado para reserva de vagas às...,1.0,1.000000,1.000000,0.978587
1,A realização de concurso público é obrigatório...,[ \nArt. 11. As normas gerais sobre concurso ...,A realização de concurso público é obrigatória...,A realização de concurso público é obrigatório...,1.0,1.000000,0.700000,0.937225
2,Quais fatores são observados durante o estágio...,"[ \nmotivo legal, hipótese em que o exercício ...",Durante o estágio probatório de um servidor pú...,"São observadas a responsabilidade, a assiduida...",1.0,1.000000,0.454545,1.000000
3,Tenho direito a folga para consulta ou exames ...,[§ 3º A jornada de trabalho em sistema de esca...,"Sim, você tem direito a folga para consultas o...","Sem prejuízo da remuneração ou subsídio, o ser...",1.0,1.000000,0.857143,0.931469
4,"Para fazer mestrado ou doutorado, tenho direit...","[ou mediante compensação de horário, afastar -...","Para realizar mestrado ou doutorado, o servido...",Pode ser concedido horário especial ao servido...,1.0,1.000000,1.000000,0.870431
5,"Fui aprovado em concurso público federal, tend...",[ \nI – as férias; \nII – as ausências previs...,"Com base no contexto fornecido, não há informa...",Pode gozar de afastamento caso haja expressa p...,1.0,0.000000,1.000000,0.000000
6,O que acontece se eu acumular dois cargos públ...,"[ \nquando houver compatibilidade de horários,...","Se você acumular dois cargos públicos, a situa...",A Lei Complementar nº 840/2011 permite a acumu...,1.0,0.666667,1.000000,0.944151
7,"Estou próximo a me aposentar, tenho alguma res...",[§ 2º A existência de dependente indicado no i...,"Com base no contexto fornecido, não há informa...",Algumas possíveis restrições: a) é necessário ...,1.0,0.400000,0.750000,0.000000
8,Posso ter um outro trabalho na esfera privada?,"[conteúdo privado, informações e outros conteú...","Com base no contexto fornecido, a possibilidad...",A Lei Complementar nº 840/2011 não proíbe o ex...,1.0,0.750000,1.000000,0.000000
9,Como é a licença prêmio/servidor?,[§ 3º O servidor candidato a cargo eletivo que...,A licença-prêmio por assiduidade é um benefíci...,A licença prêmio foi substituída pela licença ...,1.0,0.000000,1.000000,0.909327


In [ ]:
df.to_excel("RAG_Systems_Hybrid_Search_Reranking_OpenA.xlsx", index=False)